### Overview

This is the second in a series of tutorials that illustrate how to download the IRS 990 e-file data available at https://aws.amazon.com/public-data-sets/irs-990/

Specifically, in this notebook we will download into a MongoDB table the main IRS 990 filings as well as the associated "schedules."

### Load Packages

In [1]:
import sys
import time
import json

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [3]:
#http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_columns', None)  #Set PANDAS to show all columns in DataFrame
pd.set_option('max_colwidth', 500)

### MongoDB
Depending on the project, I will store the data in SQLite or MongoDB. This time I'll use MongoDB -- it's great for storing JSON data where each observation could have different variables. Before we get to the interesting part the following code blocks set up the MongoDB environment and the new database we'll be using. 

**_Note:_** In a terminal we'll have to start MongoDB by running the command *mongod* or *sudo mongod*. Then we run the following code block here to access MongoDB.

In [6]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

<br>Connect to our database that contains the filing index data we downloaded earlier.

In [7]:
# DEFINE THE MONGODB DATABASE
db = client['irs_990_db']

# DEFINE THE COLLECTIONS WHERE I'LL INSERT THE DATA
file_list_2011 = db['file_list_2011']
file_list_2012 = db['file_list_2012']
file_list_2013 = db['file_list_2013']
file_list_2014 = db['file_list_2014']
file_list_2015 = db['file_list_2015']
file_list_2016 = db['file_list_2016']
file_list_2017 = db['file_list_2017']
file_list_2018 = db['file_list_2018']
file_list_2019 = db['file_list_2019']
file_list_2020 = db['file_list_2020']
file_list_2021 = db['file_list_2021']

<br>Check how many observations in the database tables.

In [8]:
print(file_list_2011.estimated_document_count())
print(file_list_2012.estimated_document_count())
print(file_list_2013.estimated_document_count())
print(file_list_2014.estimated_document_count())
print(file_list_2015.estimated_document_count())
print(file_list_2016.estimated_document_count())
print(file_list_2017.estimated_document_count())
print(file_list_2018.estimated_document_count())
print(file_list_2019.estimated_document_count())
print(file_list_2020.estimated_document_count())
print(file_list_2021.estimated_document_count())

203075
261622
261449
387529
261034
378420
489013
457510
416910
333722
461887


<br> We will try to download all of the *990* filings and omit the *990EZ* and *990PF* filings. For example, in the 2011 index file we have 113,018 990 filings to download. We'll grab all of these as well as all those from the 2012 through 2021 index files.

In [10]:
from bson.son import SON
pipeline = [ {"$group": {"_id": "$FormType", "count": {"$sum": 1}}} ]
list(file_list_2011.aggregate(pipeline))

[{'_id': '990', 'count': 113018},
 {'_id': '990EZ', 'count': 65858},
 {'_id': '990PF', 'count': 24199}]

### Loop over List of Filings, grab e-file data, and insert into second database
First we'll write a function to turn an ordered dictionary (which is what is returned by *xmltodict*) into a normal Python dictionary so that we can combine it with the filing data gathered above.

What I'm doing in the next five or so blocks of code is looping over the *list* of 3 million plus filings that I've previously downloaded, and then downloading the full e-file 990 filings for all orgs that match one of our 8,304 EINs.

In [15]:
from json import loads, dumps
from collections import OrderedDict

def to_dict(input_ordered_dict):
    return loads(dumps(input_ordered_dict))

<br>In order to give you a sense of what we're going to do I'll set a list of two sample EINs and work through a few key steps. 

In [16]:
eins = ['340090940', '742547528']

<br>
Check how many filings there are for this sample of 2 EINs. Here we are accessing the 2021 <code>file_list</code> collection in MongoDB and printing out the two rows in the database that match the EINs in the above list. 

In [17]:
for f in file_list_2021.find({'EIN': { '$in': eins}})[:2]:
    print(f, '\n')

{'_id': ObjectId('617c359548ae60d313c44618'), 'EIN': '742547528', 'TaxPeriod': '201912', 'DLN': '93493319016300', 'FormType': '990', 'URL': 'https://s3.amazonaws.com/irs-form-990/202003199349301630_public.xml', 'OrganizationName': 'AUXILIARY TO THE PASO DEL NORTE CHILDRENS DEVELOPMENT CENTER', 'SubmittedOn': '2021-03-31', 'ObjectId': '202003199349301630', 'LastUpdated': '2021-06-11T13:10:10'} 

{'_id': ObjectId('617c359748ae60d313c67eb9'), 'EIN': '340090940', 'TaxPeriod': '201912', 'DLN': '93493321126220', 'FormType': '990', 'URL': 'https://s3.amazonaws.com/irs-form-990/202023219349312622_public.xml', 'OrganizationName': 'INTERNATIONAL BROTHERHOOD TEAMSTERS 293 TCWH', 'SubmittedOn': '2021-06-24', 'ObjectId': '202023219349312622', 'LastUpdated': '2021-09-08T16:51:44'} 



<br>To recall, our current dataset contains basic details on all 3,912,171 990 filings. We still don't have the actual 990 data, however. To get that, we will have to pick which filings we want and then access the *URL* column for that filing as seen in our dataset. What we are going to want to do later on is loop over all the rows in our database and access each filing by visiting the link shown in the *URL* field. Here we can print out the *URL* field value in the 2021 index file for our two sample EINs.

In [23]:
for file in  file_list_2020.find({'EIN': { '$in': eins}})[:2]:
    print(file['URL'], '\n')

https://s3.amazonaws.com/irs-form-990/201903199349320465_public.xml 

https://s3.amazonaws.com/irs-form-990/201903199349320485_public.xml 



<br>To access the actual Form 990 filing we are going to use a Python library called <code>urllib2</code>. In the following code block we access the first matching row in our MongoDB collection, visit the *URL* and assign the page data to a variable called *url_data*, read in the data and assign it to a variable called *f_string*, and then print out the first 1000 characters of data.  

In [26]:
import xmltodict
import urllib.request
for file in file_list_2020.find({'EIN': { '$in': eins}})[:1]:
    print(file['URL'], '\n')
    url_data = urllib.request.urlopen(file['URL'])
    f_string = url_data.read()
    print(f_string[:1000])

https://s3.amazonaws.com/irs-form-990/201903199349320465_public.xml 

b'\xef\xbb\xbf<?xml version="1.0" encoding="utf-8"?>\r\n<Return xmlns="http://www.irs.gov/efile" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.irs.gov/efile" returnVersion="2018v3.1">\r\n  <ReturnHeader binaryAttachmentCnt="0">\r\n    <ReturnTs>2019-11-15T15:12:19-06:00</ReturnTs>\r\n    <TaxPeriodEndDt>2018-12-31</TaxPeriodEndDt>\r\n    <PreparerFirmGrp>\r\n      <PreparerFirmEIN>341322309</PreparerFirmEIN>\r\n      <PreparerFirmName>\r\n        <BusinessNameLine1Txt>CIUNI &amp; PANICHI INC</BusinessNameLine1Txt>\r\n      </PreparerFirmName>\r\n      <PreparerUSAddress>\r\n        <AddressLine1Txt>25201 CHAGRIN BLVD 200</AddressLine1Txt>\r\n        <CityNm>CLEVELAND</CityNm>\r\n        <StateAbbreviationCd>OH</StateAbbreviationCd>\r\n        <ZIPCd>441225683</ZIPCd>\r\n      </PreparerUSAddress>\r\n    </PreparerFirmGrp>\r\n    <ReturnTypeCd>990</ReturnTypeCd>\r\n    <TaxPeriod

<br>The above data are in XML format. We will need to convert them to JSON format in order to insert into MongoDB. For that we will leverage the Python module <code>xmltodict</code>.

In [27]:
import xmltodict
import urllib.request
for file in file_list_2020.find({'EIN': { '$in': eins}})[:1]:
    print(file['URL'])
    url_data = urllib.request.urlopen(file['URL'])
    f_string = url_data.read()
    data = xmltodict.parse(f_string)

https://s3.amazonaws.com/irs-form-990/201903199349320465_public.xml


<br>Now the data are in a dictionary format. Let's check which keys are present.

In [28]:
data.keys()

odict_keys(['Return'])

<br>We see there is nothing relevant there, so let's drop down a level.

In [29]:
data['Return'].keys()

odict_keys(['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', '@returnVersion', 'ReturnHeader', 'ReturnData'])

<br>The first four keys do no contain much useful information.

In [30]:
data['Return']['@xmlns']

'http://www.irs.gov/efile'

In [31]:
data['Return']['@xmlns:xsi']

'http://www.w3.org/2001/XMLSchema-instance'

In [32]:
data['Return']['@xsi:schemaLocation']

'http://www.irs.gov/efile'

In [33]:
data['Return']['@returnVersion']

'2018v3.1'

<br>The <code>ReturnHeader</code> key contains 12 sub-keys. Here we have information that could be useful for certain research projects. For most purposes, however, we can skip this.

In [34]:
data['Return']['ReturnHeader'].keys()

odict_keys(['@binaryAttachmentCnt', 'ReturnTs', 'TaxPeriodEndDt', 'PreparerFirmGrp', 'ReturnTypeCd', 'TaxPeriodBeginDt', 'Filer', 'BusinessOfficerGrp', 'PreparerPersonGrp', 'FilingSecurityInformation', 'TaxYr', 'BuildTS'])

<br>Lastly, we inspect the <code>ReturnData</code> key. The first key simply provides a count of the number of documents as well as keys holding the 990 return data and any of the associated "schedules."

In [35]:
data['Return']['ReturnData'].keys()

odict_keys(['@documentCnt', 'IRS990', 'IRS990ScheduleD', 'IRS990ScheduleO', 'IRS990ScheduleR'])

<br>In almost all cases, what we want is the 990 data. As we can see here, there are 191 keys nested under the <code>IRS990</code> key. 

*Note:* - In Python 2 this is a list but in Python 3 it's a (non-iterable) *odict* so I convert it to a list first

In [37]:
print("# of keys in 990 data:", len(data['Return']['ReturnData']['IRS990'].keys()), '\n')
print("First 5 keys:", list(data['Return']['ReturnData']['IRS990'].keys())[:5])

# of keys in 990 data: 191 

First 5 keys: ['@documentId', '@referenceDocumentId', 'PrincipalOfficerNm', 'USAddress', 'GrossReceiptsAmt']


In [52]:
list(data['Return']['ReturnData']['IRS990'].keys())[:5]

['@documentId',
 '@referenceDocumentId',
 'PrincipalOfficerNm',
 'USAddress',
 'GrossReceiptsAmt']

<br>In many cases, such as <code>TotalAssetsGrp</code>, there are multiple keys nested under it. Depending on your data needs you can "flatten" these data as desired.

In [51]:
data['Return']['ReturnData']['IRS990']['TotalAssetsGrp']

OrderedDict([('BOYAmt', '594548'), ('EOYAmt', '598835')])

<br>In brief, what we want to do is write code that will loop over all relevant rows in our MongoDB collection, visit the respective URL where the 990 data are located, grab those data, convert them to a dictionary, and then insert into a new MongoDB collection. 
 

### Set up new MongoDB collection
In this second collection we will be inserting the actual 990 data we will be downloading.

In [14]:
# DEFINE MY COLLECTION (DATABASE TABLE) WHERE I'LL INSERT THE MESSAGES 
filings_990 = db['filings_990']

<br>Set a unique constraint on the collection based on *URL*. This will avert duplicates from being inserted.

In [15]:
db.filings_990.create_index([('URL', pymongo.ASCENDING)], unique=True)

In [16]:
list(db.filings_990.index_information())

['_id_', 'URL_1']

<br>There were no filings yet before first download.

In [53]:
filings_990.estimated_document_count()

### Download IRS 990 Data and Available Schedules
Now let's run the loop for all rows in all collections, grab the IRS 990 filings, and insert them into our new MongoDB table. This block has some additional code that I won't discuss in detail (see comments below for further details). The short answer is that we are looping over each row of each collection in our database, visiting the URL that contains the 990 data, and then grabbing all of the data returned by the <code>IRS990</code> key. For convenience purposes, we then combine this new data with the associated filing index data from our first database, and then insert the combined data into our new <code>filings_990</code> collection.

Note that this code block will only work for organizations that have the <code>IRS990</code> key. The check is found in the following line of code:

&nbsp; &nbsp; <code>if 'IRS990' in data['Return']['ReturnData']:</code>

This means that organizations filing *990EZ* or *990PF* will be skipped. However, the code block could easily be modified to grab 990EZ or 990PF filings. 

Compared to the <a href="http://social-metrics.org/irs-990-e-file-data-part-4/">simpler version of this tutorials</a>, we are also adding columns for five keys that are nested under the ['Return'] key. If you're not familiar with Python this line of code from the code block we'll be using will seem perplexing:

&nbsp; &nbsp; <code>return_info = {k:v for k,v in datax['Return'].iteritems() if k not in ('ReturnData')}</code>

What this code does is assign to a new dictionary called *return_info* every key and value nested under &nbsp; <code>datax['Return']</code> &nbsp;  except for <code>datax['Return]['ReturnData']</code>. &nbsp; The latter contains our 990 data, so we are going to deal with that separately. So, *return_info* will contain one column for each of the following keys: &nbsp; <code>'@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', '@returnVersion', 'ReturnHeader'</code>. &nbsp; We are not likely to use these data but it will be good to have them handy in case we need them. 

Similarly, with the following line we create a new dictionary that will contain all of the keys nested under &nbsp; <code>datax['Return']['ReturnData']</code> &nbsp; except for &nbsp; <code>datax['Return]['ReturnData']['IRS990]</code>. &nbsp; 

&nbsp; &nbsp; <code>schedules = {k:v for k,v in datax['Return']['ReturnData'].iteritems() if k not in ('IRS990')}</code>

We then combine the 9 columns from our existing MongoDB collection, the 5 columns from *return_info*, the available columns in the dictionary *schedules*, and the 200+ columns from &nbsp; <code>datax['Return]['ReturnData']['IRS990]</code>. &nbsp;  This combined dictionary is then added to our new MongoDB *filings* collection.

What we are doing here is deciding not to "flatten" the data available in the schedules. Instead, the focus of this dataset is the actual core 990 data. Our resultant dataset will have a dozen or so "background" or filing detail columns, a half-dozen or so columns containing data on any available schedules, and then 200+ columns containing the 990 data. For most research purposes this will suffice and will minimize the need to flatten keys with nested data. But whenever we have the need the data will be there waiting for us. 

<br>**_Note_**:

- This code will take a long time to download close to 2 million 990 filings. While I could have created a loop that would iterate of the 2011 through 2021 index files in turn, I have chosen to ddo this manually in the following loop. So, once you've successfully downloaded all the 990 filings in the 2011 index file, change the line that reads 
    - *for f in file_list_2011.find({'FormType': '990'}, no_cursor_timeout=True).batch_size(10)[counter:]:*
- For example, change it 2012, rerun the loop, then continue on to 2013, etc.
    - *for f in file_list_2012.find({'FormType': '990'}, no_cursor_timeout=True).batch_size(10)[counter:]:*

In [21]:
import json
import xmltodict
#import urllib3
import urllib.request
import timeit

start_time = timeit.default_timer()
from IPython.display import display, clear_output   ##### FOR USE WITH STDOUT (DYNAMIC, SINGLE-LINE PRINTING)

start_count = filings_990.estimated_document_count()
counter = 1-1
#I HAVE CHANGED THIS TO ONLY LOOP OVER THE FILES IN THE INDEX THAT ARE '990' INSTEAD OF 990PF OR 990EZ
#I STILL KEEP THE if 'IRS990' in datax BECAUSE SOME RETURNS ARE EMPTY
for f in file_list_2011.find({'FormType': '990'}, no_cursor_timeout=True).batch_size(10)[counter:]:   
    counter += 1
    print(f)
    if 'URL' in f:
        url = f['URL']
        print(url, '\n')
        try:
            url_data = urllib.request.urlopen(url)
            f_string = url_data.read()
            datax = xmltodict.parse(f_string)
        except:
            continue
        
        try:
            if 'IRS990' in datax['Return']['ReturnData']: ##### CHECK WHETHER THE 'IRS990' KEY IS PRESENT IN THE DATA
                ##### CREATE DICTIONARY CONTAINING ALL KEYS NESTED UNDER 'Return' EXCEPT FOR THE 'ReturnData' KEY
                ##### THERE WILL BE FIVE KEYS IN return_info: '@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', '@returnVersion', 'ReturnHeader'
                #####NEXT LINE IS PYTHON 2 WAY
                #return_info = {k:v for k,v in datax['Return'].iteritems() if k not in ('ReturnData')}
                return_info = {k:v for k,v in datax['Return'].items() if k not in ('ReturnData')}
                
                ##### CREATE DICTIONARY CONTAINING ALL KEYS IN ['ReturnData'] EXCEPT FOR THE 'IRS990' KEY
                ##### THERE WILL BE A KEY CALLED '@documentCount' (OR '@documentCnt') AS WELL AS ONE PER INCLUDED SCHEDULE
                #####NEXT LINE IS PYTHON 2 WAY 
                #schedules = {k:v for k,v in datax['Return']['ReturnData'].iteritems() if k not in ('IRS990')}
                schedules = {k:v for k,v in datax['Return']['ReturnData'].items() if k not in ('IRS990')}
                
                ##### CREATE DICTIONARY FOR 990 DATA
                data = datax['Return']['ReturnData']['IRS990']
                data = to_dict(data)
                        
                ##### COMBINE THE DICT OF FILING INFO FROM FIRST STEP WITH FILING DATA GATHERED HERE
                ##### NEXT LINE IS PYTHON 2; THE ONE AFTER IS MY FIRST CRACK AT PYTHON 3
                #c = {key: value for (key, value) in (f.items() + return_info.items() + schedules.items() + data.items())}
                #c = {key: value for (key, value) in dict(**f, **return_info, **schedules, **data)}
                c = c = dict(**f, **return_info, **schedules, **data)
                c.pop('_id', None)        #DROP 'id' (OR IT WILL NOT INSERT)
                t = json.dumps(c)
                #print t 
                loaded_entry = json.loads(t) 
                #print type(loaded_entry) , loaded_entry    #<type 'dict'>
                try:
                    filings_990.insert_one(loaded_entry)
                except pymongo.errors.DuplicateKeyError:# , e:
                    print("*****THERE IS A DUPLICATEKEYERROR*****")#(e, '\n')
        
        except KeyError:
            print('IRS9990 key not in data (likely a 990EZ or 990PF filer)')
            #print(data['Return']['ReturnData'].keys())
            print(data.keys())
              
    else:
        print(f['IsAvailable'])
         
    clear_output()
    print(('counter: ', counter, f['OrganizationName']), '\n')
    sys.stdout.flush()        
        
elapsed = timeit.default_timer() - start_time
print('# of minutes: ', elapsed/60, '\n')
print("Total # of filings in dnatabase:", filings_990.estimated_document_count())
print("# of filings added to database:", filings_990.estimated_document_count()  - start_count, '\n')

('counter: ', 199946, 'CASA OF EL PASO INC') 

# of minutes:  1048.0079850383336 

Total # of filings in dnatabase: 2016601
# of filings added to database: 0 



('# of minutes: ', 1.6810495116666668)
('Number of columns:', 1)
('Number of observations:', 1617926)


,URL
0,https://s3.amazonaws.com/irs-form-990/201100709349300510_public.xml


### Generate and save a list of downloaded 990 filings
There is a possibility that not all of the filings downloaded. Accordingly, we are going to create a list of all filings that were successfully inserted into our new collection. As a first step, we will create a PANDAS dataframe, called ``df``,  into which we will insert all of the *URL* fields from our *filings_990* collection.

In [ ]:
%%time
df = pd.DataFrame(list(filings_990.find({}, {'URL':1, 
    '_id':0})))
elapsed = timeit.default_timer() - start_time
print("Number of columns:", len(df.columns))
print("Number of observations:", len(df))
df[:1]

In [54]:
downloaded = df['URL'].tolist()
print len(downloaded)
downloaded[:2]

In [24]:
import json
with open('downloaded_990_filings.json', 'w') as fp:
    json.dump(downloaded, fp)

### Rerun download on non-downloaded filings
Run this on all index files from 2011 through 2021. Manually change ``for f in file_list....`` line as you did above.

In [36]:
import json
import xmltodict
import urllib2
import timeit

start_time = timeit.default_timer()
from IPython.display import display, clear_output   ##### FOR USE WITH STDOUT (DYNAMIC, SINGLE-LINE PRINTING)

start_count = filings_990.count_documents({})
counter = 1-1

for f in file_list_2011.find({}, no_cursor_timeout=True).batch_size(10)[counter:]:    
    counter += 1
    
    print f
    if 'URL' in f:
        url = f['URL']
        print url, '\n'
        if url not in downloaded:
            print 'Original -- will try to download>>>>>>>>>>>>'
            url_data = urllib2.urlopen(url)
            f_string = url_data.read()
            datax = xmltodict.parse(f_string)
        
            try:
                if 'IRS990' in datax['Return']['ReturnData']: ##### CHECK WHETHER THE 'IRS990' KEY IS PRESENT IN THE DATA
                    ##### CREATE DICTIONARY CONTAINING ALL KEYS NESTED UNDER 'Return' EXCEPT FOR THE 'ReturnData' KEY
                    ##### THERE WILL BE FIVE KEYS IN return_info: '@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', '@returnVersion', 'ReturnHeader'
                    return_info = {k:v for k,v in datax['Return'].iteritems() if k not in ('ReturnData')}
                    ##### CREATE DICTIONARY CONTAINING ALL KEYS IN ['ReturnData'] EXCEPT FOR THE 'IRS990' KEY
                    ##### THERE WILL BE A KEY CALLED '@documentCount' (OR '@documentCnt') AS WELL AS ONE PER INCLUDED SCHEDULE
                    schedules = {k:v for k,v in datax['Return']['ReturnData'].iteritems() if k not in ('IRS990')}
                    ##### CREATE DICTIONARY FOR 990 DATA
                    data = datax['Return']['ReturnData']['IRS990']
                    data = to_dict(data)
                        
                    ##### COMBINE THE DICT OF FILING INFO FROM FIRST STEP WITH FILING DATA GATHERED HERE
                    c = {key: value for (key, value) in (f.items() + return_info.items() + schedules.items() + data.items())}
                    c.pop('_id', None)        #DROP 'id' (OR IT WILL NOT INSERT)
                    t = json.dumps(c)
                    #print t 
                    loaded_entry = json.loads(t) 
                  #print type(loaded_entry) , loaded_entry    #<type 'dict'>
                    try:
                        filings_990.insert_one(loaded_entry)
                    except pymongo.errors.DuplicateKeyError, e:
                        print e, '\n'
        
            except KeyError:
                print 'IRS9990 key not in data (likely a 990EZ or 990PF filer)'
                print data['Return']['ReturnData'].keys()
        else:
            print '....FILING HAS ALREADY BEEN DOWNLOADED>>>>>'
             
    else:
        print f['IsAvailable']
         
    clear_output()
    print ('counter: ', counter, f['OrganizationName']), '\n'
    sys.stdout.flush()        
        
elapsed = timeit.default_timer() - start_time
print '# of minutes: ', elapsed/60, '\n'
print "# of filings added to database:", filings_990.count_documents({}) - start_count, '\n'              

('counter: ', 2763336, u'GLOBAL MEDIA OUTREACH') 

# of minutes:  659.30174687 

# of filings added to database: 0 



<br>As noted above, our code will only work for organizations that have the <code>IRS990</code> key. We won't get data for 990EZ or 990PF filers. Focusing on one of the three groups of filers would be the typical research design decision.